In [1]:
!pip uninstall scikit-learn -y
!pip uninstall xgboost -y
!pip install scikit-learn==1.3.2
!pip install xgboost==1.7.0
!pip install  -U autogluon

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: xgboost 2.0.3
Uninstalling xgboost-2.0.3:
  Successfully uninstalled xgboost-2.0.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 83.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tpot 0.12.1 requires xgboost>=1.1.0, which is not installed.
nilearn 0.11.1 requires scikit-learn>=1.4.0, but you have scikit-learn 1.3.2 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
ERROR: Could not find a version that satisfies the requirement xgboost==1.7.0 (from versions: 0.4a12, 0.4a13, 0.4a14, 0.4a15, 0.4a18, 0.4a19, 0.4a20, 0.4a21, 0.4a22, 0.4a23, 0.4a24, 0.4a25, 0.4a26, 0.4a27, 0.4a28, 0.4a29, 0.4a30, 0.6a1, 0.6a2, 0.7.post3,

In [2]:
# Imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/calories-burnt-prediction/calories.csv
/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv


In [4]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv')
display(train_df.head())
display(train_df.shape)

# In the sidebar include the Calories Burnt Prediction dataset to access this.
orig_df = pd.read_csv('/kaggle/input/calories-burnt-prediction/calories.csv')
orig_df.rename(columns={'User_ID':'id','Gender':'Sex'}, inplace=True)
display(orig_df.head())
display(orig_df.shape)

# Combine the two datasets
train_df = pd.concat([train_df, orig_df], axis=0, ignore_index=True)
train_df = train_df.drop_duplicates()
display(train_df.shape)

# Check for missing values
display(train_df.isnull().sum())

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


(750000, 9)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,14733363,male,68,190.0,94.0,29.0,105.0,40.8,231.0
1,14861698,female,20,166.0,60.0,14.0,94.0,40.3,66.0
2,11179863,male,69,179.0,79.0,5.0,88.0,38.7,26.0
3,16180408,female,34,179.0,71.0,13.0,100.0,40.5,71.0
4,17771927,female,27,154.0,58.0,10.0,81.0,39.8,35.0


(15000, 9)

(765000, 9)

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [5]:
test_df = pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv')

In [6]:
# Feature Augmentation
# https://www.kaggle.com/code/khsamaha/single-xgboost-model-predicting-calories-py
for df in [train_df, test_df]:
    df['BMI'] = df['Weight']/((df['Height'] /100) **2)
    df['Intensity'] = df['Duration'] * df['Heart_Rate']
    df['Body_Temp_vs_Duration'] = df['Body_Temp'] * df['Duration']
    df['Weight_vs_Duration'] = df['Weight'] / df['Duration']
    df['Age_vs_Heart_Rate'] = df['Age'] / df['Heart_Rate']
    df['Age_vs_Weight'] = df['Age'] / df['Weight']
    df['Age_vs_Duration'] = df['Age'] / df['Duration']
    df['Weight_vs_Heart_Rate'] = df['Weight'] / df['Heart_Rate']

In [7]:
# This is the metric that is used to evaluate the performance of the model in the competition
# But may not work well for ensemble.
"""
def rmsle(y_true, y_pred):
    # Ensure predictions are non-negative
    y_pred = np.maximum(0, y_pred)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

rmsle_scorer = make_scorer(
    name='rmsle',
    score_func=rmsle,
    optimum=0,  # Lower is better for RMSLE
    greater_is_better=False,
    needs_proba=False
)
"""

"\ndef rmsle(y_true, y_pred):\n    # Ensure predictions are non-negative\n    y_pred = np.maximum(0, y_pred)\n    return np.sqrt(mean_squared_log_error(y_true, y_pred))\n\nrmsle_scorer = make_scorer(\n    name='rmsle',\n    score_func=rmsle,\n    optimum=0,  # Lower is better for RMSLE\n    greater_is_better=False,\n    needs_proba=False\n)\n"

In [8]:
# Use RMSE as error metric. Instead of calories, optimize on log(Calories)
train_df['log_Calories'] = np.log1p(train_df['Calories'])
train_df = train_df.drop(columns=['Calories'])

In [9]:
# AutoML-Gluon to take care of the best stack of ML models 
from autogluon.core.metrics import make_scorer
from autogluon.tabular import TabularPredictor

automl_predictor = TabularPredictor(
                                  label='log_Calories', 
                                  eval_metric="root_mean_squared_error",
                                  problem_type="regression"
                                 )

automl_predictor.fit(train_df,
                     presets='best_quality',
                     time_limit=3600 * 4,
                     keep_only_best=True)

No path specified. Models will be saved in: "AutogluonModels/ag-20250507_021942"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.06 GB / 31.35 GB (95.9%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be 

(_ray_fit pid=659) [1000]	valid_set's rmse: 0.0595841
(_ray_fit pid=659) [2000]	valid_set's rmse: 0.059058 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=659) [3000]	valid_set's rmse: 0.0588651 [repeated 4x across cluster]
(_ray_fit pid=659) [4000]	valid_set's rmse: 0.0588116 [repeated 4x across cluster]
(_ray_fit pid=657) [5000]	valid_set's rmse: 0.0606299 [repeated 4x across cluster]
(_ray_fit pid=657) [6000]	valid_set's rmse: 0.0606001 [repeated 4x across cluster]
(_ray_fit pid=657) [7000]	valid_set's rmse: 0.0605968 [repeated 4x across cluster]
(_ray_fit pid=657) [8000]	valid_set's rmse: 0.0606031 [repeated 3x across cluster]
(_ray_fit pid=660) [9000]	valid_set's rmse: 0.0600959 [repeated 3x across cluster]
(_ray_fit pid=861) [1000]	valid_set's rmse: 0.0617546
(_ray_fit pid=66

(_dystack pid=305) 	-0.0599	 = Validation score   (-root_mean_squared_error)
(_dystack pid=305) 	932.92s	 = Training   runtime
(_dystack pid=305) 	589.65s	 = Validation runtime
(_dystack pid=305) Fitting model: LightGBM_BAG_L1 ... Training model for up to 1392.22s of the 2590.43s of remaining time.
(_dystack pid=305) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=1.85%)


(_ray_fit pid=1213) [1000]	valid_set's rmse: 0.0588448
(_ray_fit pid=1215) [2000]	valid_set's rmse: 0.0602606 [repeated 4x across cluster]
(_ray_fit pid=1360) [1000]	valid_set's rmse: 0.060167 [repeated 2x across cluster]
(_ray_fit pid=1395) [1000]	valid_set's rmse: 0.0590662
(_ray_fit pid=1432) [1000]	valid_set's rmse: 0.0591842
(_ray_fit pid=1468) [1000]	valid_set's rmse: 0.0613494 [repeated 2x across cluster]
(_ray_fit pid=1360) [3000]	valid_set's rmse: 0.0599395
(_ray_fit pid=1468) [2000]	valid_set's rmse: 0.0611527
(_ray_fit pid=1360) [4000]	valid_set's rmse: 0.0599501
(_ray_fit pid=1468) [3000]	valid_set's rmse: 0.0611581


(_dystack pid=305) 	-0.0598	 = Validation score   (-root_mean_squared_error)
(_dystack pid=305) 	222.97s	 = Training   runtime
(_dystack pid=305) 	70.24s	 = Validation runtime
(_dystack pid=305) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 1155.60s of the 2353.81s of remaining time.
(_dystack pid=305) /usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
(_dystack pid=305)   warnings.warn(
(_dystack pid=305) 	-0.0621	 = Validation score   (-root_mean_squared_error)
(_dystack pid=305) 	608.77s	 = Training   runtime
(_dystack pid=305) 	19.88s	 = Validation runtime
(_dystack pid=305) Fitting model: CatBoost_BAG_L1 ... Training model for up to 521.44s of the 1719.66s of remaining time.
(_dystack pid=305) 	Fitting 8 child models (S1F1 - S1F8

(_ray_fit pid=2586) [1000]	valid_set's rmse: 0.0595146
(_ray_fit pid=2585) [1000]	valid_set's rmse: 0.0606902
(_ray_fit pid=2588) [2000]	valid_set's rmse: 0.0615469 [repeated 3x across cluster]
(_ray_fit pid=2587) [3000]	valid_set's rmse: 0.0586167 [repeated 4x across cluster]
(_ray_fit pid=2586) [4000]	valid_set's rmse: 0.0592053 [repeated 4x across cluster]
(_ray_fit pid=2586) [5000]	valid_set's rmse: 0.0592327 [repeated 4x across cluster]
(_ray_fit pid=2772) [1000]	valid_set's rmse: 0.0595999 [repeated 2x across cluster]
(_ray_fit pid=2847) [1000]	valid_set's rmse: 0.0604297 [repeated 2x across cluster]
(_ray_fit pid=2772) [2000]	valid_set's rmse: 0.0594513 [repeated 2x across cluster]
(_ray_fit pid=2847) [2000]	valid_set's rmse: 0.0603128 [repeated 2x across cluster]
(_ray_fit pid=2801) [3000]	valid_set's rmse: 0.0611225 [repeated 3x across cluster]
(_ray_fit pid=2847) [3000]	valid_set's rmse: 0.0603093
(_ray_fit pid=2848) [3000]	valid_set's rmse: 0.0595497
(_ray_fit pid=2847) [400

(_dystack pid=305) 	-0.06	 = Validation score   (-root_mean_squared_error)
(_dystack pid=305) 	544.54s	 = Training   runtime
(_dystack pid=305) 	218.06s	 = Validation runtime
(_dystack pid=305) Fitting model: LightGBM_BAG_L2 ... Training model for up to 620.00s of the 619.92s of remaining time.
(_dystack pid=305) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=2.24%)
(_dystack pid=305) 	-0.0592	 = Validation score   (-root_mean_squared_error)
(_dystack pid=305) 	48.09s	 = Training   runtime
(_dystack pid=305) 	5.01s	 = Validation runtime
(_dystack pid=305) Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 568.30s of the 568.22s of remaining time.
(_dystack pid=305) 	Warning: Reducing model 'n_estimators' from 300 -> 114 due to low time. Expected time usage reduced from 1488.5s -> 567.7s...
(_dystack pid=305) /usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEsti

In [10]:
automl_predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.058578,root_mean_squared_error,1871.462719,6529.776999,0.012187,2.550095,2,True,9
1,CatBoost_BAG_L1,-0.058967,root_mean_squared_error,2.263970,1694.145616,2.263970,1694.145616,1,True,2
2,CatBoost_r177_BAG_L1,-0.058998,root_mean_squared_error,1.717753,1073.076093,1.717753,1073.076093,1,True,7
3,LightGBM_r131_BAG_L1,-0.059387,root_mean_squared_error,998.014301,924.538813,998.014301,924.538813,1,True,8
4,LightGBMLarge_BAG_L1,-0.059521,root_mean_squared_error,79.048541,183.390453,79.048541,183.390453,1,True,6
5,LightGBMXT_BAG_L1,-0.059741,root_mean_squared_error,696.308633,1123.275011,696.308633,1123.275011,1,True,1
6,XGBoost_BAG_L1,-0.059814,root_mean_squared_error,66.872329,249.789056,66.872329,249.789056,1,True,5
7,NeuralNetFastAI_BAG_L1,-0.060408,root_mean_squared_error,5.845558,1049.935006,5.845558,1049.935006,1,True,4
8,ExtraTreesMSE_BAG_L1,-0.060904,root_mean_squared_error,21.379448,229.076857,21.379448,229.076857,1,True,3


In [11]:
automl_predictor.save("/kaggle/working/autogluon_rmse_model")

In [12]:
y_test = automl_predictor.predict(test_df)
submission = pd.DataFrame({
    "id": test_df["id"],
    "Calories": np.expm1(y_test)
})

submission.to_csv("submission.csv", index=False)